# Imports

In [1]:
from WS_Mdl.utils import Up_log, Pa_WS, INI_to_d, get_elapsed_time_str, get_MdlN_paths

In [2]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
from pathlib import Path
import re
import pandas as pd
from colored import fg, bg, attr
import tempfile
import shutil as sh
from filelock import FileLock as FL
import subprocess as sp
from datetime import datetime as DT
from multiprocessing import cpu_count
import warnings
from multiprocessing import Pool
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._read_only")

In [3]:
from matplotlib import pyplot as plt

In [4]:
from pprint import pprint as PP

In [5]:
import imod
from imod import msw as MSW

In [6]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM

In [7]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)

<module 'WS_Mdl.utils_imod' from 'C:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\WS_Mdl\\utils_imod.py'>

In [8]:
IL.reload(imod)

<module 'imod' from 'c:\\mamba_envs\\WS\\Lib\\site-packages\\imod\\__init__.py'>

# Options + Basics

In [9]:
MdlN = 'Nbr31'

In [10]:
U.set_verbose(False)

In [11]:
d_Pa = U.get_MdlN_paths(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])

In [12]:
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]

# Read PRJ

In [13]:
PRJ_content = UIM.read_PRJ_with_OBS(Pa_PRJ)[0]

# Load PRJ

In [14]:
PRJ_, PRJ_OBS = UIM.open_PRJ_with_OBS(Pa_PRJ)

In [15]:
PRJ = PRJ_[0]

In [16]:
period_data = PRJ_[1]

In [17]:
PRJ.keys()

dict_keys(['cap', 'extra', 'bnd', 'top', 'bot', 'khv', 'kva', 'shd', 'sto', 'spy', 'hfb-1', 'hfb-2', 'hfb-3', 'hfb-4', 'hfb-5', 'hfb-6', 'hfb-7', 'hfb-8', 'hfb-9', 'hfb-10', 'hfb-11', 'hfb-12', 'hfb-13', 'hfb-14', 'hfb-15', 'hfb-16', 'hfb-17', 'hfb-18', 'hfb-19', 'hfb-20', 'hfb-21', 'hfb-22', 'hfb-23', 'hfb-24', 'hfb-25', 'hfb-26', 'hfb-27', 'hfb-28', 'hfb-29', 'hfb-30', 'hfb-31', 'hfb-32', 'hfb-33', 'hfb-34', 'hfb-35', 'hfb-36', 'hfb-37', 'hfb-38', 'hfb-39', 'hfb-40', 'hfb-41', 'hfb-42', 'hfb-43', 'hfb-44', 'hfb-45', 'hfb-46', 'hfb-47', 'hfb-48', 'hfb-49', 'hfb-50', 'hfb-51', 'hfb-52', 'hfb-53', 'hfb-54', 'hfb-55', 'hfb-56', 'hfb-57', 'hfb-58', 'hfb-59', 'hfb-60', 'hfb-61', 'hfb-62', 'hfb-63', 'hfb-64', 'hfb-65', 'hfb-66', 'hfb-67', 'hfb-68', 'hfb-69', 'hfb-70', 'hfb-71', 'hfb-72', 'hfb-73', 'hfb-74', 'hfb-75', 'hfb-76', 'hfb-77', 'hfb-78', 'hfb-79', 'hfb-80', 'hfb-81', 'hfb-82', 'hfb-83', 'hfb-84', 'hfb-85', 'hfb-86', 'hfb-87', 'hfb-88', 'hfb-89', 'hfb-90', 'hfb-91', 'hfb-92', 'hfb-9

In [18]:
period_data

{}

# Prepare Sim

## 1st attempt

In [19]:
times = pd.date_range(SP_date_1st, SP_date_last, freq='D')

In [20]:
# Convert IMOD5 to MODFLOW6 (without CAP package due to mixed grid issues)
PRJ_no_cap = PRJ.copy()
PRJ_CAP, PRJ_extra = {}, {}
if "cap" in PRJ_no_cap:
    PRJ_CAP['cap'] = PRJ_no_cap["cap"]
    PRJ_extra['extra'] = PRJ_no_cap['extra']
    del PRJ_no_cap["cap"], PRJ_no_cap['extra']
    print("Removed CAP package due to mixed grid compatibility issues")

Sim_MF6 = imod.mf6.Modflow6Simulation.from_imod5_data(PRJ_no_cap, period_data, times)
print("Simulation created successfully!")

Removed CAP package due to mixed grid compatibility issues
Simulation created successfully!


We have to remove MetaSWAP (MSW), otherwie this function fails.

In [21]:
PRJ_CAP.keys()

dict_keys(['cap'])

In [22]:
PRJ_CAP['cap'].keys()

dict_keys(['urban_infiltration_capacity', 'rootzone_thickness', 'soil_moisture_fraction', 'meteo_station_number', 'rural_infiltration_capacity', 'perched_water_table_level', 'artificial_recharge_capacity', 'soil_physical_unit', 'urban_ponding_depth', 'rural_ponding_depth', 'landuse', 'rural_runoff_resistance', 'surface_elevation', 'artificial_recharge_layer', 'urban_runoff_resistance', 'rural_runon_resistance', 'wetted_area', 'urban_area', 'conductivitiy_factor', 'boundary', 'artificial_recharge', 'urban_runon_resistance'])

In [23]:
MF6_DIS = Sim_MF6["imported_model"]["dis"]

In [24]:
MSW_Mdl = MSW.MetaSwapModel.from_imod5_data(PRJ_CAP, MF6_DIS, times)

KeyError: 'extra'

## 2nd attempt

Let's investigate this error:

In [26]:
PRJ_CAP['cap'].keys()

dict_keys(['urban_infiltration_capacity', 'rootzone_thickness', 'soil_moisture_fraction', 'meteo_station_number', 'rural_infiltration_capacity', 'perched_water_table_level', 'artificial_recharge_capacity', 'soil_physical_unit', 'urban_ponding_depth', 'rural_ponding_depth', 'landuse', 'rural_runoff_resistance', 'surface_elevation', 'artificial_recharge_layer', 'urban_runoff_resistance', 'rural_runon_resistance', 'wetted_area', 'urban_area', 'conductivitiy_factor', 'boundary', 'artificial_recharge', 'urban_runon_resistance'])

In [ ]:
# Debug the grid alignment issue
print("=== Target Discretization ===")
print(f"Type: {type(target_dis)}")
print(f"Keys: {list(target_dis.keys()) if hasattr(target_dis, 'keys') else 'No keys method'}")

# Access the data from target_dis
if hasattr(target_dis, 'dataset'):
    print(f"Dataset keys: {list(target_dis.dataset.keys())}")
    target_x = target_dis.dataset.x.values
    target_y = target_dis.dataset.y.values
else:
    # Try accessing as a dataset directly
    target_x = target_dis.x.values
    target_y = target_dis.y.values

print(f"Target X range: {target_x.min()} to {target_x.max()}, length: {len(target_x)}")
print(f"Target Y range: {target_y.min()} to {target_y.max()}, length: {len(target_y)}")

print("\n=== CAP Data ===")
sample_cap = PRJ_CAP['cap']['urban_area']
cap_x = sample_cap.x.values
cap_y = sample_cap.y.values
print(f"CAP X range: {cap_x.min()} to {cap_x.max()}, length: {len(cap_x)}")
print(f"CAP Y range: {cap_y.min()} to {cap_y.max()}, length: {len(cap_y)}")

print(f"\n=== Comparison ===")
print(f"X grids compatible: {len(target_x) == len(cap_x) and np.allclose(target_x, cap_x)}")
print(f"Y grids compatible: {len(target_y) == len(cap_y) and np.allclose(target_y, cap_y)}")

In [ ]:
# Remove 'extra' from cap if it exists (to avoid isel error on metadata)
if 'extra' in PRJ_CAP['cap'].keys():
    PRJ_CAP['cap']['extra']

# Get the MODFLOW6 extent (target domain)
target_dis_orig = Sim_MF6["imported_model"]["dis"]
mf6_x_min, mf6_x_max = target_dis_orig.dataset.x.min().item(), target_dis_orig.dataset.x.max().item()
mf6_y_min, mf6_y_max = target_dis_orig.dataset.y.min().item(), target_dis_orig.dataset.y.max().item()

print(f"MODFLOW6 domain: X({mf6_x_min:.1f} to {mf6_x_max:.1f}), Y({mf6_y_min:.1f} to {mf6_y_max:.1f})")

# Get CAP grid info
sample_cap = PRJ_CAP['cap']['urban_area']
cap_dx = abs(sample_cap.x.values[1] - sample_cap.x.values[0])  # CAP resolution (absolute value)
cap_dy = abs(sample_cap.y.values[1] - sample_cap.y.values[0])  # CAP resolution (absolute value)

print(f"CAP grid resolution: dx={cap_dx:.1f}m, dy={cap_dy:.1f}m")

# Create exact coordinates for the refined grid within MODFLOW6 domain
refined_x = np.arange(mf6_x_min + cap_dx/2, mf6_x_max, cap_dx)
refined_y = np.arange(mf6_y_max - cap_dy/2, mf6_y_min, -cap_dy)

print(f"Refined grid: x from {refined_x.min():.1f} to {refined_x.max():.1f} ({len(refined_x)} cells)")
print(f"Refined grid: y from {refined_y.min():.1f} to {refined_y.max():.1f} ({len(refined_y)} cells)")

# Create a refined target discretization
target_dis_refined = target_dis_orig.dataset.interp(
    x=refined_x,
    y=refined_y,
    method='nearest'
)
target_dis_refined['idomain'] = target_dis_refined['idomain'].astype(int)

# Convert back to StructuredDiscretization
from imod.mf6 import StructuredDiscretization
target_dis = StructuredDiscretization(
    idomain=target_dis_refined['idomain'],
    top=target_dis_refined['top'],
    bottom=target_dis_refined['bottom']
)

print(f"Refined target_dis shape: {target_dis.dataset.sizes}")

# Now regrid all CAP data to the exact refined coordinates
PRJ_CAP_regridded = {'cap': {}}
for key, data in PRJ_CAP['cap'].items():
    print(f"Processing {key}: dims = {data.dims}")
    
    # Check if data has spatial dimensions (x, y)
    if 'x' in data.dims and 'y' in data.dims:
        if key == 'wetted_area':
            # For area-related fields, we need to preserve the total area
            # The wetted_area represents the actual area, which should remain as cell area
            regridded_data = data.interp(x=refined_x, y=refined_y, method='nearest')
            # Set to full cell area (25m x 25m = 625 m²)
            PRJ_CAP_regridded['cap'][key] = regridded_data * 0 + (cap_dx * cap_dy)
        else:
            # For other spatial data, interpolate normally
            PRJ_CAP_regridded['cap'][key] = data.interp(
                x=refined_x,
                y=refined_y,
                method='nearest'
            )
    else:
        # Keep non-spatial data as is
        PRJ_CAP_regridded['cap'][key] = data

print("CAP data regridding completed.")

# Merge with extra metadata
PRJ_CAP_for_MSW = {**PRJ_CAP_regridded, **PRJ_extra}

# Create the MetaSwap model
MSW_Mdl = MSW.MetaSwapModel.from_imod5_data(PRJ_CAP_for_MSW, target_dis, times)
print("🎉 MetaSwap model created successfully!")

# Load MSW

In [ ]:
MSW_Mdl = MSW.MetaSwapModel(unsaturated_database=PJ(Pa_WS, r'data\MetaSWAP_DB\LHM2018_v02v'))

# Junkyard

#### Test sim without cap

 <!-- # Simple solution: Remove CAP package and test if conversion works
 print("=== Testing Without CAP Package ===")

 # Create a copy of PRJ without the CAP package
 PRJ_no_cap = PRJ.copy()
 if "cap" in PRJ_no_cap:
     del PRJ_no_cap["cap"]
     print("CAP package removed from PRJ")
 else:
     print("CAP package not found in PRJ")

 print(f"Original PRJ keys: {list(PRJ.keys())}")
 print(f"PRJ without CAP keys: {list(PRJ_no_cap.keys())}")

 # Test the conversion without CAP
 try:
     print("\n=== Testing Simulation Conversion Without CAP ===")
     simulation = imod.mf6.Modflow6Simulation.from_imod5_data(PRJ_no_cap, period_data, times)
     print("SUCCESS: Simulation created without CAP package!")
     print(f"Simulation keys: {list(simulation.keys())}")
    
     # Check what packages were created
     if "imported_model" in simulation:
         model = simulation["imported_model"]
         print(f"Model packages: {list(model.keys())}")
        
 except Exception as e:
     print(f"Error without CAP: {e}")
     import traceback
     traceback.print_exc() -->

#### Investigate well error

from imod.mf6.wel import LayeredWell, Well

PRJ['wel-WEL_Br_Wa_T_NBr1']['layer']

Well.from_imod5_data('wel-WEL_Br_Wa_T_NBr1', PRJ, times)

os.remove(Pa_PRJ_temp)  # Delete temp PRJ file as it's not needed anymore.